# Bellingham Stormwater Monitoring Analysis

In [369]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
from IPython.display import display, HTML

# Define the CSS to set the height of the output container
css_style = """
<style>
    div.output_area {
        height: 800px; /* Adjust this value based on your needs */
        overflow-y: auto; /* Vertical scroll */
        overflow-x: auto; /* Horizontal scroll */
    }
</style>
"""
# Apply the CSS style to the notebook
display(HTML(css_style))

pd.set_option('display.max_rows', 1000) 
pd.set_option('display.max_columns', 1000) 

## TODO
1. Review the other site data as well as Bellingham
2. Determine treatment of duplicates (DONE)
3. Add F chart to show variability.  Review F-Chart calcs from stats book.  Not std
4. Try altair plots (DONE)
5. Research how e.coli data is collected
6. What is variability with time and incubation?
7. Verify that all these tests were taken with similar methods (R-Card, lab, ...).  Comments refer to counts on some measurements which allude to R-Card
8. Convert to plots using Tidy dataset frame and not stack (plotly, or Altair) (DONE)
9. Add column called is_exceedence and calculate counts (see below)
10. Look into stormwater bacteria level modelling
11. Investigate log-normal distribution for defects



![Bacteria](e.coli-sampling-protocol.png) 

In [371]:
#Original dataset provided by Kirsten McDade on May 28th 2024
dataset1 = 'Salish Sea Stormwater Monitoring Database-20240528.csv'

#Updated dataset provided by Kirsten McDade on Oct 8th 2024
dataset2 = 'Bellingham SW Data_ALL_20241008.csv'

data = pd.read_csv(dataset2)

## Clean raw data

### Clean Dataset1 specific

# Remove 'Unnamed' columns
data = data.loc[:, ~data.columns.str.contains('^Unnamed')].dropna(how='all')

# Remove rows with Nan in 'Sample Date' column
data = data.dropna(subset=['Sample Date'])
data = data.rename(columns={'E. Coli': 'E-Coli'}) #This column name causes data plotting weirdness

data.head()

## Clean Dataset2 Specific

In [ ]:
data = data.rename(columns={'Survey::Survey_Date': 'Sample Date'})
data = data.rename(columns={'Survey::City': 'City'})
data = data.rename(columns={'Outfall_ID': 'Site ID'})
data = data.rename(columns={'E. coli at 100ml': 'E-Coli'})
data = data.rename(columns={'Enterococcus at 100ml': 'Enterococcus'})

# Convert Sample Date to datetime
data['Sample Date'] = pd.to_datetime(data['Sample Date'])
#data['Sample Date'] = pd.to_datetime(data['Sample Date'], errors='coerce', format='%m-%d-%y')

#Remove rows with NaN E-Coli values
data = data.dropna(subset=['E-Coli'])

data.dtypes
data.describe()

#data

In [ ]:
#show unique site ids
site_ids = data['Site ID'].unique()
site_ids

In [374]:
# Rename Site Id 'Cedar' to 'Cedar St'
data['Site ID'] = data['Site ID'].replace('Cedar', 'Cedar St')

In [ ]:
# Identify duplicates
duplicates = data[data.duplicated(keep=False)]

data['is_duplicate'] = data.duplicated(subset=['Site ID', 'Sample Date'], keep=False)

# Remove rows with Nan in 'Sample Date' column
#data = data.dropna(subset=['Sample Date'])

data[data['is_duplicate'] == True].describe()
#data[data['is_duplicate'] == True]
#data

In [376]:
# Add a column called is_execeedence to identify if the E-Coli or Enterococcus values exceed the threshold
bacteria_threshold = 310
data['bacteria_threshold'] = bacteria_threshold

data['is_exceedence'] = False
data.loc[data['E-Coli'] > bacteria_threshold, 'is_exceedence'] = True
data.loc[data['Enterococcus'] > bacteria_threshold, 'is_exceedence'] = True

In [ ]:
data.columns


In [378]:

#dataset1 specific
#ecoli = data[['Sample Date', 'Site ID', 'E-Coli', 'Comments', 'is_duplicate', 'is_exceedence', 'bacteria_threshold']].copy()
#dataset2 specific
ecoli = data[['Sample Date', 'Site ID', 'E-Coli', 'is_duplicate', 'is_exceedence', 'bacteria_threshold']].copy()

# Extract the month for aggregation in ecoli
ecoli['Month'] = ecoli['Sample Date'].dt.to_period('M')  # Convert date to month period for grouping

# Re-format the 'Month' column to a proper datetime format
#ecoli['Month'] = pd.to_timestamp(ecoli['Month'])

#dataset 1 specific
#enterococcus = data[['Sample Date', 'Site ID', 'Enterococcus', 'Comments', 'is_duplicate', 'is_exceedence', 'bacteria_threshold']].copy()
#dataset2 specific
enterococcus = data[['Sample Date', 'Site ID', 'Enterococcus', 'is_duplicate', 'is_exceedence', 'bacteria_threshold']].copy()


# Extract the month for aggregation in ecoli
enterococcus['Month'] = enterococcus['Sample Date'].dt.to_period('M')  # Convert date to month period for grouping

# Sort data by date
ecoli = ecoli.sort_values(['Sample Date', 'Site ID'])
enterococcus = enterococcus.sort_values(['Sample Date', 'Site ID'])

#ecoli.dtypes
#ecoli[ecoli['is_duplicate'] == True]
#ecoli

Save Dataframe for use in GPT optimization

In [379]:
ecoli.to_csv("ecoli.csv")

In [380]:
ecoli.site_id = 'Bennett Ave'

## Clean Data

In [381]:

ecoli_data_clean = ecoli.dropna(subset=['E-Coli']).copy()

# Re-format the 'Month' column to a proper datetime format and then to string
ecoli_data_clean['Month'] = ecoli_data_clean['Month'].dt.to_timestamp()

ecoli_data_clean = ecoli_data_clean.sort_values(['Site ID',  'Sample Date'])

#ecoli_data_clean[ecoli_data_clean['Month'] == '2021-09-01']
#ecoli_data_clean

In [ ]:
import scipy.stats as stats
from matplotlib.ticker import MultipleLocator

# Calculate statistics
ecoli_stats = ecoli_data_clean['E-Coli'].dropna().describe(percentiles=[.25, .5, .75])
median = ecoli_stats['50%']
q1 = ecoli_stats['25%']
q3 = ecoli_stats['75%']
iqr = q3 - q1



plt.show()

#plot a box and whisker plot of the e-coli collumn of the ecoli_data_clean dataframe
plt.figure(figsize=(10, 6))
plt.boxplot(ecoli_data_clean['E-Coli'].dropna(), vert=True, patch_artist=True)
plt.title('Box and Whisker Plot of E-Coli Levels')
plt.xlabel('E-Coli')
plt.ylabel('cfu/100ml')
plt.ylim(0, 1000)  # Set y-axis limit
plt.grid(True)

# Annotate the plot with median, Q1, Q3, and IQR
arrowprops = dict(facecolor='black', shrink=0.05, width=0.5, headwidth=4)
plt.annotate(f'Median: {median:.2f}', xy=(1, median), xytext=(1.1, median),
             arrowprops=arrowprops)
plt.annotate(f'Q1: {q1:.2f}', xy=(1, q1), xytext=(1.1, q1),
             arrowprops=arrowprops)
plt.annotate(f'Q3: {q3:.2f}', xy=(1, q3), xytext=(1.1, q3),
             arrowprops=arrowprops)
plt.annotate(f'IQR: {iqr:.2f}', xy=(1, q3), xytext=(1.1, q3 + 50),
             arrowprops=arrowprops)

plt.show()

#plot a box and whisker plot of the e-coli collumn of the ecoli_data_clean dataframe
plt.figure(figsize=(10, 6))
plt.boxplot(ecoli_data_clean['E-Coli'][ecoli_data_clean['E-Coli'] < 1000].dropna(), vert=True, patch_artist=True)
plt.title('Box and Whisker Plot of E-Coli Levels')
plt.xlabel('E-Coli')
plt.ylabel('cfu/100ml')
plt.grid(True)
plt.show()

# Plot a cumulative probability plot of the 'E-Coli' column
plt.figure(figsize=(10, 6))
stats.probplot(ecoli_data_clean['E-Coli'].dropna(), dist="norm", plot=plt)
plt.title('Cumulative Probability Plot of E-Coli Levels')
plt.xlabel('Theoretical Quantiles')
plt.ylabel('Ordered Values')
plt.grid(True)
plt.show()

# Plot a cumulative probability plot of the filtered 'E-Coli' column
plt.figure(figsize=(10, 6))
stats.probplot(ecoli_data_clean['E-Coli'][ecoli_data_clean['E-Coli'] < 1000].dropna(), dist="norm", plot=plt)
plt.title('Cumulative Probability Plot of E-Coli Levels (Values > 1000)')
plt.xlabel('Theoretical Quantiles')
plt.ylabel('Ordered Values')
plt.grid(True)
plt.gca().yaxis.set_major_locator(MultipleLocator(500))  # Set y-axis grid interval to 100
plt.show()

In [ ]:
print(ecoli_data_clean.describe())
print(ecoli_data_clean['E-Coli'][ecoli_data_clean['E-Coli'] < 1000].dropna().describe())

## Filter and Aggregate Data
Concerntrate on recent data > 2021

Remove outliers which skew the baseline

Aggregate data for the same month using geometric mean

In [384]:
#ecoli_data_clean

In [385]:
from scipy.stats import gmean

#Aggregate data for the same month using geometric mean
# Group by 'Site ID' and 'Month' and calculate geometric mean
grouped_data = ecoli_data_clean.groupby(['Site ID', 'Month']).agg({
    'E-Coli': lambda x: gmean(x[x > 0]) if (x > 0).any() else 0  # Geometric mean, ignoring non-positive values
}).reset_index()

#grouped_data[grouped_data['Month'] == '2021-09-01']
#grouped_data.duplicated(subset=['Site ID', 'Month'])
#grouped_data

Determine the appropriate range for CFU/ml.  Counts above 250 are considered Too Numerous To Count (TNTC) because it is impossible to tell whether colonies are separated (https://wic.oregonstate.edu/microbiology-writing-guide-presenting-data)

In [ ]:
start_date = pd.Timestamp('2021-01-01')
#filtered_data = ecoli_data_clean[ecoli_data_clean['Month'] >= start_date]
filtered_data = grouped_data

#Filter for site id = Bennett Ave  
#filtered_data = filtered_data[filtered_data['Site ID'] == 'Bennett Ave']

#Filter e-coli values greater than 5000
#filtered_data = filtered_data[filtered_data['E-Coli'] <= 5000]

# Remove rows where Site ID is 'G05-1002', 'G06-1001', or 'G11-1001'
filtered_data = filtered_data[~filtered_data['Site ID'].isin(['G05-1002', 'G06-1001', 'G11-1001'])]

#show unique site ids
site_ids = filtered_data['Site ID'].unique()
site_ids

#filtered_data


What's up with G06-1001, G05-1002, and G11-1011 site ids?

In [ ]:
filtered_data.dtypes
numeric_columns = filtered_data.select_dtypes(include=[np.number]).columns #Only group numeric columns or error
median_per_month = filtered_data.groupby(pd.Grouper(key='Month', freq='MS'))[numeric_columns].median().reset_index() #'MS' = Month Start 
#average_per_month = average_per_month.sort_values(['Site ID', 'Site ID'])

median_per_month

## Visualize

In [ ]:
import altair as alt

# Create a selection for the dropdown
#site_id_dropdown = alt.binding_select(options=filtered_data['Site ID'].unique(), name='Site ID')
#site_id_select = alt.selection_point(fields=['Site ID'], bind=site_id_dropdown, value={'Site ID': filtered_data['Site ID'].unique()[0]})

# Ensure the data is specified at the top level
plot_data = median_per_month

# Base chart
chart = alt.Chart(plot_data).mark_line(point=True).encode(
    x=alt.X('Month:T', timeUnit='yearmonth', axis=alt.Axis(format='%m-%Y', title='Sample Date', labelAngle=90)),
    y=alt.Y('E-Coli:Q', scale=alt.Scale(domain=[0, 2000])),  # Set y-axis limit
    #color='Site ID:N',
    tooltip=['Month:T', 'E-Coli:Q']
).properties(
    title='Median E. Coli Measurements by Month',
    width=800,
    height=400
#).add_selection(
#    site_id_select
#).transform_filter(
#    site_id_select
).interactive()

# Horizontal line at y=320
horizontal_line = alt.Chart(pd.DataFrame({'y': [320]})).mark_rule(
    color='red',
    strokeDash=[5, 5]
).encode(
    y='y:Q'
)

line_chart = chart + horizontal_line

# Histogram of E-Coli
histogram = alt.Chart(filtered_data).mark_bar().encode(
    x=alt.X('E-Coli:Q', bin=alt.Bin(step=50), scale=alt.Scale(domain=[0, 3000])),
    y='count()',
    #color='Site ID:N'
).properties(
    title='Histogram of E. Coli Measurements',
    width=800,
    height=200
)

# Combine the line chart and histogram
final_chart = alt.vconcat(line_chart, histogram)

final_chart.show()



In [ ]:
# Ensure the data is specified at the top level
plot_data = filtered_data
# Filter data to include only the specified site IDs
#plot_data = plot_data[plot_data['Site ID'].isin(['Bennett Ave', 'Broadway St', 'C St'])]
#plot_data = plot_data[plot_data['Site ID'].isin(['Cedar St', 'Little Squalicum Creek', 'Olive St'])]
#plot_data = plot_data[plot_data['Site ID'].isin(['Padden Creek', 'Squalicum Creek', 'Whatcom Creek', 'Willow St'])]

# Base chart
base_chart = alt.Chart().mark_line(point=True).encode(
    x=alt.X('Month:T', timeUnit='yearmonth', axis=alt.Axis(format='%m-%Y', title='Sample Date', labelAngle=90)),
    y=alt.Y('E-Coli:Q', scale=alt.Scale(domain=[0, 5000])),  # Set y-axis limit
    color='Site ID:N',
    tooltip=['Month:T', 'Site ID:N', 'E-Coli:Q']
).properties(
    width=400,
    height=200
)

# Horizontal line at y=320
horizontal_line = alt.Chart(pd.DataFrame({'y': [bacteria_threshold]})).mark_rule(
    color='red',
    strokeDash=[5, 5]
).encode(
    y='y:Q'
)

# Layer the base chart and the horizontal line
layered_chart = alt.layer(base_chart, horizontal_line, data=plot_data)

# Apply faceting after layering with max columns
faceted_chart = layered_chart.facet(
    column='Site ID:N'
).properties(
    title='E. Coli Measurements by Month for Each Site ID'
).interactive()

faceted_chart.show()

In [ ]:
# Ensure the data is specified at the top level
plot_data = filtered_data
# Filter data to include only the specified site IDs
#plot_data = plot_data[plot_data['Site ID'].isin(['Bennett Ave', 'Broadway St', 'C St'])]
#plot_data = plot_data[plot_data['Site ID'].isin(['Cedar St', 'Little Squalicum Creek', 'Olive St'])]
#plot_data = plot_data[plot_data['Site ID'].isin(['Padden Creek', 'Squalicum Creek', 'Whatcom Creek', 'Willow St'])]

# Base chart
base_chart = alt.Chart(plot_data).mark_line(point=True).encode(
    x=alt.X('Month:T', timeUnit='yearmonth', axis=alt.Axis(format='%m-%Y', title='Sample Date', labelAngle=90)),
    y=alt.Y('E-Coli:Q', scale=alt.Scale(domain=[0, 1000], clamp=True), axis=alt.Axis(tickCount=10)),  # Set y-axis limit
    color='Site ID:N',
    tooltip=['Month:T', 'Site ID:N', 'E-Coli:Q'],
    facet = alt.Facet('Site ID:N', columns=4)
).properties(
    width=400,
    height=200
)

base_chart

In [ ]:
# Ensure the data is specified at the top level
plot_data = filtered_data
# Filter data to include only the specified site IDs
#plot_data = plot_data[plot_data['Site ID'].isin(['Bennett Ave', 'Broadway St', 'C St'])]
#plot_data = plot_data[plot_data['Site ID'].isin(['Cedar St', 'Little Squalicum Creek', 'Olive St'])]
#plot_data = plot_data[plot_data['Site ID'].isin(['Padden Creek', 'Squalicum Creek', 'Whatcom Creek', 'Willow St'])]

# Base chart
base_chart = alt.Chart(plot_data).mark_line(point=True).encode(
    x=alt.X('Month:T', timeUnit='yearmonth', axis=alt.Axis(format='%m-%Y', title='Sample Date', labelAngle=90)),
    y=alt.Y('E-Coli:Q', scale=alt.Scale(type='log', domain=[1, 30000], clamp=True), axis=alt.Axis(tickCount=10)),  # Set y-axis to log scale
    color='Site ID:N',
    tooltip=['Month:T', 'Site ID:N', 'E-Coli:Q'],
    facet = alt.Facet('Site ID:N', columns=4)
).properties(
    width=400,
    height=200
)

base_chart

In [ ]:
#https://www.geeksforgeeks.org/understanding-facet-wrap-in-altair/ 

#Second Filter e-coli values greater than 1000
filtered_data2 = filtered_data[filtered_data['E-Coli'] <= 1000]

# Histogram of E-Coli with bin size 50, faceted by Site ID
histogram = alt.Chart(filtered_data2).mark_bar().encode(
    x=alt.X('E-Coli:Q', bin=alt.Bin(step=50)),
    y='count()',
    color='Site ID:N',
    facet=alt.Facet('Site ID:N', columns=4)
).properties(
    title='E. Coli Histogram',
    width=200,
    height=200
)

#.facet(
#    column='Site ID:N',
#)

histogram.show()

## Modelling Typical Stormwater bacteria levels

In [ ]:
import pandas as pd
import numpy as np

# Set random seed for reproducibility
np.random.seed(42)

# Parameters for dataset
num_samples = 500  # Total data points across all outfalls
outfall_ids = ["Outfall_1", "Outfall_2", "Outfall_3"]  # Outfall locations

# Calculate samples per outfall to ensure uniform distribution
samples_per_outfall = num_samples // len(outfall_ids)  
total_samples = samples_per_outfall * len(outfall_ids)  # Adjust to align

# Generate timestamps for each sample, spaced by the hour
times = pd.date_range("2024-01-01", periods=samples_per_outfall, freq="H")

# Generate bacteria levels using a log-normal distribution
bacteria_levels = np.random.lognormal(mean=3.5, sigma=1.0, size=total_samples)

# Repeat outfall IDs and tile timestamps to align with the samples
outfalls = np.repeat(outfall_ids, samples_per_outfall)
timestamps = np.tile(times, len(outfall_ids))

# Create the DataFrame
data = pd.DataFrame({
    "Timestamp": timestamps,
    "Outfall": outfalls,
    "Bacteria_CFU_100mL": bacteria_levels
})

# Display the dataset (only needed for your reference)
print(data.head())



In [ ]:
import matplotlib.pyplot as plt

# Filter the data for Outfall_1
outfall_to_plot = "Outfall_1"
subset = data[data["Outfall"] == outfall_to_plot]

# Plot the histogram of bacteria levels
plt.figure(figsize=(10, 6))
plt.hist(subset["Bacteria_CFU_100mL"], bins=30, edgecolor='black', color='orange')
plt.title(f"Histogram of Bacteria Levels at {outfall_to_plot}")
plt.xlabel("Bacteria Levels (CFU/100mL)")
plt.ylabel("Frequency")
plt.show()

### Matplotlib Visulazation

In [ ]:
import matplotlib.dates as mdates

site_data = filtered_data[filtered_data['Site ID'] == 'Bennett Ave']

# Calculate the moving average (window size of 5 for example)
site_data['E-Coli Moving Average'] = site_data['E-Coli'].rolling(window=4).mean()

fig, ax = plt.subplots(figsize=(16, 8))
for site_id, group in site_data.groupby('Site ID'):
    ax.plot(group['Month'], group['E-Coli'], marker='o', linestyle='-', label=f'Site {site_id}')
    ax.plot(group['Month'], group['E-Coli Moving Average'], marker='', linestyle='-', linewidth=3, color='orange', label=f'Moving Average {site_id}')

# Set the major locator and formatter for the x-axis to show every month
ax.xaxis.set_major_locator(mdates.MonthLocator())
ax.xaxis.set_major_formatter(mdates.DateFormatter('%b %Y'))

# Add vertical grid lines for each month
ax.grid(which='major', axis='x', linestyle='--', linewidth=0.5)

ax.set_title('Raw E. coli Readings by Site Over Time - Bennett Ave')
ax.set_xlabel('Month')
ax.set_ylabel('E. Coli (cfu/100ml)')
ax.axhline(y=bacteria_threshold, color='red', linewidth=2, linestyle='--', label='Threshold (310 cfu/100ml)')
ax.legend(title='Site ID')
plt.xticks(rotation=45)  # Rotate x-axis labels for better visibility
plt.tight_layout()
plt.show()

#site_data

Bennett Ave Data has mnay outliers 

In [ ]:
# Create a figure with two subplots (ax for raw readings and ax2 for variability F-chart)
fig, (ax, ax2) = plt.subplots(nrows=2, ncols=1, figsize=(16, 16), sharex=True)

# Plotting E. Coli readings by Site
for site_id, group in ecoli.groupby('Site ID'):
    ax.plot(group['Sample Date'], group['E-Coli'], marker='o', linestyle='-', label=f'Site {site_id}')

ax.set_title('Raw E. coli Readings by Site Over Time')
ax.set_ylabel('E. Coli (cfu/100ml)')
ax.axhline(y=310, color='red', linestyle='--', label='Threshold (310 cfu/100ml)')
ax.legend(title='Site ID')
plt.xticks(rotation=45)  # Rotate x-axis labels for better visibility

# Plotting F-chart of variability (Standard Deviation over time)
# Calculating standard deviation for each date
std_dev_by_date = ecoli.groupby('Sample Date')['E-Coli'].std()
ax2.plot(std_dev_by_date.index, std_dev_by_date, marker='o', linestyle='-', color='green')

# Plotting F-chart of variability (Standard Deviation over time) by Site ID

#for site_id, group in ecoli.groupby('Site ID'):
#    std_dev = group.groupby('Sample Date')['E. Coli'].std()
#    ax2.plot(std_dev.index, std_dev, marker='o', linestyle='-', label=f'Site {site_id}')

ax2.set_title('Standard Deviation of E. Coli Readings by Site Over Time')
ax2.set_xlabel('Sample Date')
ax2.set_ylabel('Standard Deviation (cfu/100ml)')
ax2.legend(title='Site ID')

plt.tight_layout()
plt.show()

In [ ]:
ecoli[ecoli['Site ID'] == 'Bennett Ave']

In [398]:
# Aggregate data for plotting
ecoli_aggregated_mean = ecoli.groupby(['Month', 'Site ID'])['E-Coli'].mean().unstack()
ecoli_aggregated_std = ecoli.groupby(['Month', 'Site ID'])['E-Coli'].std().unstack()
enterococcus_aggregated = enterococcus.groupby(['Month', 'Site ID'])['Enterococcus'].mean().unstack()

In [ ]:
ecoli_aggregated_mean
#ecoli_aggregated[ecoli_aggregated['Site ID'] == 'Bennett Ave']

In [400]:
ecoli_threshold = 310
enterococcus_threshold = 500

In [ ]:
fig, ax = plt.subplots(2, 1, figsize=(16, 12), sharex=True)
ecoli_aggregated_mean.clip(upper=5000).plot(ax=ax[0], marker='o', linestyle='-', title='E. Coli Readings by Site Over Time with Clamped Values')
ax[0].set_ylabel('E. Coli (cfu/100ml)')
ax[0].axhline(y=ecoli_threshold, color='red', linestyle='--', label='E. Coli Threshold (cfu/100ml)')
ecoli_aggregated_std.plot(ax=ax[1], marker='o', linestyle='-', title='Std Dev of E. Coli Readings')

In [ ]:
# Plotting the control charts
fig, ax = plt.subplots(2, 1, figsize=(16, 12), sharex=True)
ecoli_aggregated_mean.clip(upper=5000).plot(ax=ax[0], marker='o', linestyle='-', title='E. Coli Readings by Site Over Time with Clamped Values')
ax[0].set_ylabel('E. Coli (cfu/100ml)')
ax[0].axhline(y=ecoli_threshold, color='red', linestyle='--', label='E. Coli Threshold (cfu/100ml)')
enterococcus_aggregated.plot(ax=ax[1], marker='o', linestyle='-', title='Enterococcus Readings by Site Over Time')
ax[1].set_ylabel('Enterococcus (cfu/100ml)')
ax[1].axhline(y=enterococcus_threshold, color='red', linestyle='--', label='Enterococcus Threshold (cfu/100ml)')
plt.tight_layout()
plt.show()

In [ ]:
ecoli_aggregated_mean = ecoli_aggregated_mean[ecoli_aggregated_mean.index >= '2023-01']
enterococcus_aggregated_mean = enterococcus_aggregated[enterococcus_aggregated.index >= '2023-01']

fig, ax = plt.subplots(2, 1, figsize=(16, 12), sharex=True)
ecoli_aggregated_mean.clip(upper=5000).plot(ax=ax[0], marker='o', linestyle='-', title='E. Coli Readings by Site Over Time')
ax[0].set_ylabel('E. Coli (cfu/100ml)')
ax[0].axhline(y=ecoli_threshold, color='red', linestyle='--', label='E. Coli Threshold (cfu/100ml)')
enterococcus_aggregated.plot(ax=ax[1], marker='o', linestyle='-', title='Enterococcus Readings by Site Over Time')
ax[1].set_ylabel('Enterococcus (cfu/100ml)')
ax[1].axhline(y=enterococcus_threshold, color='red', linestyle='--', label='Enterococcus Threshold (cfu/100ml)')

plt.tight_layout()
plt.show()

In [ ]:
# Generate histograms
fig, ax = plt.subplots(figsize=(12, 8))
colors = plt.cm.viridis(np.linspace(0, 1, len(ecoli_aggregated_mean.columns)))

for (site, values), color in zip(ecoli_aggregated_mean.items(), colors):
    ax.hist(values.dropna(), bins=20, color=color, alpha=0.6, edgecolor='black', label=site)

ax.axvline(x=ecoli_threshold, color='red', linestyle='--', label='Threshold (310 cfu/100ml)')
ax.set_title('E. Coli Distribution Across Sites')
ax.set_xlabel('E. Coli (cfu/100ml)')
ax.set_ylabel('Frequency')
ax.legend(title='Site ID')
plt.show()

In [ ]:
# Calculate exceedances for E. coli
ecoli_exceedances = ecoli[ecoli['E-Coli'] > ecoli_threshold]
ecoli_exceedances['Year'] = ecoli_exceedances['Sample Date'].dt.year
exceedance_counts = ecoli_exceedances.groupby(['Year', 'Site ID']).size().reset_index(name='Exceedance Count')
total_counts = ecoli.groupby([ecoli['Sample Date'].dt.year, 'Site ID']).size().reset_index(name='Total Readings')
total_counts.rename(columns={'Sample Date': 'Year'}, inplace=True)
exceedance_data = pd.merge(total_counts, exceedance_counts, on=['Year', 'Site ID'], how='left').fillna(0)
exceedance_data['Exceedance Percentage'] = (exceedance_data['Exceedance Count'] / exceedance_data['Total Readings']) * 100

print(exceedance_data)